In [1]:
import pandas as pd
import numpy as np

from tqdm.notebook import tqdm

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

%matplotlib inline

In [2]:
links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')

In [3]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

In [4]:
movies_modified = movies.copy()

In [5]:
movies_modified['list_genres'] = [change_string(g) for g in movies_modified.genres.values]
movies_modified 

,movieId,title,genres,list_genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy,Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance,Comedy Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy,Comedy
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,Action Animation Comedy Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,Animation Comedy Fantasy
9739,193585,Flint (2017),Drama,Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,Action Animation


In [6]:
l = movies_modified.merge(tags)
p = l.groupby('movieId').agg({'tag':list}).reset_index()
p['tag'] = [' '.join(map(str, l)) for l in p['tag']]
p

,movieId,tag
0,1,pixar pixar fun
1,2,fantasy magic board game Robin Williams game
2,3,moldy old
3,5,pregnancy remake
4,7,remake
...,...,...
1567,183611,Comedy funny Rachel McAdams
1568,184471,adventure Alicia Vikander video game adaptation
1569,187593,Josh Brolin Ryan Reynolds sarcasm
1570,187595,Emilia Clarke star wars


In [7]:
movies_modified_new = movies_modified.join(p.set_index('movieId'), on='movieId')
movies_modified_new = movies_modified_new.fillna(' ')
movies_modified_new

,movieId,title,genres,list_genres,tag
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,Adventure Animation Children Comedy Fantasy,pixar pixar fun
1,2,Jumanji (1995),Adventure|Children|Fantasy,Adventure Children Fantasy,fantasy magic board game Robin Williams game
2,3,Grumpier Old Men (1995),Comedy|Romance,Comedy Romance,moldy old
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,Comedy Drama Romance,
4,5,Father of the Bride Part II (1995),Comedy,Comedy,pregnancy remake
...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,Action Animation Comedy Fantasy,
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,Animation Comedy Fantasy,
9739,193585,Flint (2017),Drama,Drama,
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,Action Animation,


In [31]:
tfidf_transformer_genres = TfidfVectorizer()
tfidf_transformer_genres.fit(movies_modified_new['list_genres'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [32]:
tfidf_transformer_tag = TfidfVectorizer()
tfidf_transformer_tag.fit(movies_modified_new['tag'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [33]:
with_ratings = movies_modified_new.merge(ratings)
p = ratings.groupby('movieId')['rating'].median().reset_index()
k = ratings.groupby('movieId')['rating'].mean().reset_index()
p = p.rename(columns={'rating': 'rating_median'})
k = k.rename(columns={'rating': 'rating_mean'})
with_ratings = with_ratings.merge(p)
with_ratings = with_ratings.merge(k)
with_ratings = with_ratings[['movieId', 'title', 'list_genres', 'tag', 'userId', 'rating', 'rating_median', 'rating_mean']]
with_ratings

,movieId,title,list_genres,tag,userId,rating,rating_median,rating_mean
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,pixar pixar fun,1,4.0,4.0,3.92093
1,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,pixar pixar fun,5,4.0,4.0,3.92093
2,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,pixar pixar fun,7,4.5,4.0,3.92093
3,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,pixar pixar fun,15,2.5,4.0,3.92093
4,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy,pixar pixar fun,17,4.5,4.0,3.92093
...,...,...,...,...,...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),Action Animation Comedy Fantasy,,184,4.0,4.0,4.00000
100832,193583,No Game No Life: Zero (2017),Animation Comedy Fantasy,,184,3.5,3.5,3.50000
100833,193585,Flint (2017),Drama,,184,3.5,3.5,3.50000
100834,193587,Bungo Stray Dogs: Dead Apple (2018),Action Animation,,184,3.5,3.5,3.50000


In [34]:
for_model_df = with_ratings[with_ratings['userId'] == 100]
for_model_df

,movieId,title,list_genres,tag,userId,rating,rating_median,rating_mean
337,3,Grumpier Old Men (1995),Comedy Romance,moldy old,100,3.5,3.00,3.259615
759,11,"American President, The (1995)",Comedy Drama Romance,politics president,100,4.0,4.00,3.671429
882,16,Casino (1995),Crime Drama,Mafia,100,4.5,4.00,3.926829
966,17,Sense and Sensibility (1995),Drama Romance,Jane Austen,100,4.5,4.00,3.776119
1056,19,Ace Ventura: When Nature Calls (1995),Comedy,,100,1.0,3.00,2.727273
...,...,...,...,...,...,...,...,...
72254,6785,Seven Brides for Seven Brothers (1954),Comedy Musical Romance Western,Lonesome Polecat,100,4.5,3.75,3.750000
72599,6873,Intolerable Cruelty (2003),Comedy Romance,,100,4.0,3.00,3.175000
73819,7149,Something's Gotta Give (2003),Comedy Drama Romance,,100,3.5,4.00,3.647059
76290,8529,"Terminal, The (2004)",Comedy Drama Romance,,100,4.0,3.50,3.319149


In [44]:
X_1 = pd.DataFrame(tfidf_transformer_genres.transform(for_model_df['list_genres']).toarray())
X_2 = pd.DataFrame(tfidf_transformer_tag.transform(for_model_df['list_genres']).toarray())
y =  for_model_df['rating']

In [84]:
for_model_df = for_model_df.reset_index()
X = pd.concat([X_1, X_2, for_model_df['rating_median'],for_model_df['rating_mean']],sort=False,axis=1)
X

,0,1,2,3,4,5,6,7,8,9,...,1736,1737,1738,1739,1740,1741,1742,1743,rating_median,rating_mean
0,0.00000,0.000000,0.0,0.0,0.570915,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.00,3.259615
1,0.00000,0.000000,0.0,0.0,0.505015,0.000000,0.0,0.466405,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.00,3.671429
2,0.00000,0.000000,0.0,0.0,0.000000,0.863943,0.0,0.503590,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.00,3.926829
3,0.00000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.540377,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.00,3.776119
4,0.00000,0.000000,0.0,0.0,1.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.00,2.727273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,0.00000,0.000000,0.0,0.0,0.260029,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.75,3.750000
144,0.00000,0.000000,0.0,0.0,0.570915,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.00,3.175000
145,0.00000,0.000000,0.0,0.0,0.505015,0.000000,0.0,0.466405,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.00,3.647059
146,0.00000,0.000000,0.0,0.0,0.505015,0.000000,0.0,0.466405,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.50,3.319149


In [89]:
from sklearn.decomposition import PCA

In [101]:
pca = PCA(n_components=2)
pca.fit(X)
X = pca.transform(X)

In [97]:
X_tain, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 12)

In [98]:
lr = LinearRegression()
lr.fit(X_tain,y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [102]:
from sklearn.metrics import mean_squared_error
import math

In [103]:
MSE = mean_squared_error(y_test, lr.predict(X_test))
RMSE = math.sqrt(MSE)
RMSE

0.6432127919318268